# Color Identification in Images using OpenCV and LAB Color Space

This notebook identifies colors in an image using OpenCV. The color matching is done using the LAB color space for improved perceptual accuracy.

In [ ]:
# Importing the required libraries
import cv2
import pandas as pd
import numpy as np

## Loading Colors Data from CSV

In [ ]:
# Function to load colors from a CSV file
def load_colors(csv_path):
    columns = ['ColorName', 'Hex', 'R', 'G', 'B']
    colors = pd.read_csv(csv_path, names=columns)
    return colors

# Load colors
colors_path = 'Colors.csv'  # Path to your CSV file
colors_df = load_colors(colors_path)
colors_df.head()

## Improved Color Matching Function using LAB Color Space

In [ ]:
# Function to get the closest color name
def get_color_name(R, G, B, colors_df):
    # Convert RGB to LAB (CIELAB) color space for better perceptual matching
    color_diff = []
    for index, row in colors_df.iterrows():
        lab1 = cv2.cvtColor(np.uint8([[row[['B', 'G', 'R']]]]), cv2.COLOR_BGR2LAB)
        lab2 = cv2.cvtColor(np.uint8([[[B, G, R]]]), cv2.COLOR_BGR2LAB)
        delta_e = np.linalg.norm(lab1 - lab2)  # Euclidean distance in LAB space
        color_diff.append((delta_e, row['ColorName']))
    
    # Get the closest color
    closest_color = min(color_diff, key=lambda x: x[0])
    return closest_color[1]

## Image Loading and Mouse Interaction

In [ ]:
# Load image
image_path = 'Image.jpg'  # Path to your image file
image = cv2.imread(image_path)

# Variables for storing coordinates and color values
clicked = False
r = g = b = x_pos = y_pos = 0

def draw_function(event, x, y, flags, param):
    global clicked, r, g, b, x_pos, y_pos
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked = True
        x_pos = x
        y_pos = y
        b, g, r = image[y, x]  # Get BGR values

cv2.namedWindow('Improved Color Detection')
cv2.setMouseCallback('Improved Color Detection', draw_function)

## Displaying Color Information on Image

In [ ]:
while True:
    cv2.imshow('Improved Color Detection', image)
    if clicked:
        # Draw a rectangle with the detected color
        cv2.rectangle(image, (20, 20), (750, 60), (int(b), int(g), int(r)), -1)
        # Get the color name
        color_name = get_color_name(r, g, b, colors_df)
        text = f'{color_name} R={r} G={g} B={b}'
        
        # Set text color for visibility
        text_color = (0, 0, 0) if r+g+b > 500 else (255, 255, 255)
        cv2.putText(image, text, (50, 50), 2, 0.8, text_color, 2, cv2.LINE_AA)
        clicked = False

    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()